## GAN의 전성시대를 연 DCGAN(Deep Convolutional GAN)
GAN은 학습이 불안정하기로 악명이 높다. 학습이 어렵다는 점은 GAN 모델이 다양한 곳에 응용되는 것을 가로막는 큰 장애물이었다. 이런 상황에서 수많은 실험 끝에 안정적인 학습이 가능한 GAN 모델의 구조를 찾아낸 것이 DCGAN이다.

DCGAN의 특징은 몇 가지로 요약할 수 있다. 먼저, 선형 레이어와 풀링 레이어(Pooling Layer)를 최대한 배제하고 합성곱(Convolution)과 ‘Transposed Convolution(Fractional-Strided Convolution)’으로 네트워크 구조를 만들었다. 풀링 레이어는 여러 딥러닝 모델에서 불필요한 매개변수의 수를 줄이고 중요한 특징만을 골라내는 역할을 하는 레이어지만 이미지의 위치 정보를 잃어버린다는 단점이 있다. 이미지를 생성하기 위해서는 위치 정보가 중요하기 때문에 DCGAN은 풀링 레이어를 배제했다. 선형 레이어 역시 마찬가지로 위치 정보를 잃어버리므로 모델의 깊은 레이어에서는 선형 레이어를 사용하지 않았다.

DCGAN의 또 다른 특징은 배치 정규화(Batch Normalization)를 사용했다는 점이다. 배치 정규화는 레이어의 입력 데이터 분포가 치우쳐져 있을 때 평균과 분산을 조정해주는 역할을 한다. 이는 역전파가 각 레이어에 쉽게 전달되도록 해 학습이 안정적으로 이뤄지도록 돕는 데 중요한 역할을 한다.

이외에도 DCGAN은 수많은 실험을 통해 GAN을 학습시키는 가장 좋은 조건들을 찾아냈다. DCGAN은 마지막 레이어를 제외하고 생성자의 모든 레이어에 ReLU를 사용했고, 구분자의 모든 레이어에 LeakyReLU를 사용했다. 또한, 가장 좋은 최적화 기법과 적절한 학습 속도(Learning Rate) 등을 찾아내기도 했다.

DCGAN의 성공은 GAN 모델이 유명해지는 데 결정적인 역할을 했다. DCGAN에서 사용한 모델 구조는 아직도 새로운 GAN 모델을 설계할 때 베이스 모델이 되고 있다.

DCGAN의 네트워크 구조는 기존 GAN에서 생성자와 구분자만 교체하는 것만으로 간단히 구현할 수 있다. DCGAN의 생성자는 GAN과 마찬가지로 랜덤 벡터 z를 받고 가짜 이미지를 생성하는 함수다. 다만 그 구현에서 ‘Transposed Convolution’과 배치 정규화 등을 사용한다는 점이 다르다.

## <코드9> DCGAN의 생성자

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

/Users/janghan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
class Generator(nn.Module):
    
  # 네트워크 구조
  def __init__(self):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
      nn.ConvTranspose2d(in_channels=100, out_channels=28*8, 
        kernel_size=7, stride=1, padding=0, 
        bias=False),
      nn.BatchNorm2d(num_features=28*8),
      nn.ReLU(inplace=True),
      nn.ConvTranspose2d(in_channels=28*8, out_channels=28*4, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*4),
      nn.ReLU(True),
      nn.ConvTranspose2d(in_channels=28*4, out_channels=1, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.Tanh()
    )
        
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
  def forward(self, inputs):
    inputs = inputs.view(-1, 100, 1, 1)
    return self.main(inputs)

DCGAN의 구분자도 GAN의 구분자와 입력과 출력이 동일하다. 단지 convolution 레이어와 배치 정규화 등을 사용한다는 차이점만 있다.

## <코드10> DCGAN의 구분자

In [6]:
class Discriminator(nn.Module):
    
  # 네트워크 구조
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=28*4, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*4),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(in_channels=28*4, out_channels=28*8, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*8),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(in_channels=28*8, out_channels=1, 
        kernel_size=7, stride=1, padding=0, 
        bias=False),
      nn.Sigmoid()
    )
        
  # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
  # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
  def forward(self, inputs):
    o = self.main(inputs)
    return o.view(-1, 1)

DCGAN의 또 다른 혁신은 학습이 잘 이뤄졌는지 확인하기 위한 여러 가지 검증 방법을 도입했다는 점이다. 그 중 하나가 잠재 공간에 실제 데이터의 특성이 투영됐는지 살펴보는 것이다. 예를 들어 사람 얼굴을 생성하는 모델이 잘 학습되면 단순히 사람 얼굴을 잘 만들어내는 것뿐만 아니라 성별, 머리 색깔, 얼굴방향, 선글라스를 썼는지 여부 등 의미 있는 단위들이 잠재 공간에 드러나게 된다. 즉 생성자의 입력인 100차원짜리 ‘z’ 벡터의 값을 바꿔보는것으로 생성자의 출력인 이미지의 속성을 바꿔볼 수 있다는 것이다.

예를 들어 잠재공간에서 얼굴 방향에 해당하는 특성을 찾아낼 수 있고, ‘z’ 벡터에서 이에 해당하는 값을 바꿈으로써 이미지에서 얼굴이 바라보고 있는 방향을 바꿔볼수 있다. 이것이 가능하다는 것은 생성자가 얼굴의 의미적인 속성을 학습했다는 것을 뜻한다.

## 이미지를 새로운 이미지로 변형하는 cGAN
때때로 이미지를 처음부터 생성하기보다 이미 있는 이미지를 다른 영역의 이미지로 변형하고 싶은 경우가 많다. 예를 들어, 스케치에 채색하거나, 흑백 사진을 컬러로 만들거나, 낮 사진을 밤 사진으로 바꾸고 싶을 때 등이다. ‘cGAN(Conditional GAN)’은 이를 가능케 해주는 모델이다.

기존의 GAN의 생성자가 랜덤 벡터를 입력으로 받는 것에 비해 cGAN의 생성자는 변형할 이미지를 입력으로 받는다. 그 뒤 생성자는 입력 이미지에 맞는 변형된 이미지를 출력한다. 예를 들어 스케치 사진을 받은 생성자는 그 스케치에 맞는 색을 칠한 뒤 채색된 이미지를 출력하는 것이다. 구분자는 스케치와 채색된 이미지를 모두 보고 그 채색된 이미지가 과연 스케치에 어울리는지 판단한다. 구분자를 속이기 위해서 생성자는 첫째, 진짜 같은 이미지를 만들어야 하고 둘째, 스케치에 맞는 이미지를 만들어야 한다.

그림9: cGAN의 생성자와 구분자

cGAN의 혁신은 주어진 이미지를 새로운 이미지로 변형하는 수많은 문제를 하나의 간단한 네트워크 구조로 모두 풀었다는 점이다. 모든 문제는 이미지에서 의미적인 정보를 찾아내어 다른 이미지로 바꾸는 문제로 볼 수 있기 때문이다. 이렇게 한 영역의 이미지를 다른 영역의 이미지로 변형하는 문제의 경우 cGAN이 유용하게 쓰일 수 있다.

그림10: cGAN으로 가능한 이미지 처리 예시

## 다양한 종류의 GAN
앞서 소개한 모델 외에도 GAN의 성능을 높이고 새로운 분야에 응용하려는 다양한 종류의 GAN 모델이 있다. ‘WGAN(Wasserstein GAN)’은 GAN에서 실제 데이터 분포와 근사하는 분포가 얼마나 다른지 측정하는 거리 개념을 바꿔 안정적인 학습을 가능하게 만들었다. ‘EBGAN(Energy-based GAN)’은 GAN을 에너지 관점에서 바라봄으로써 역시 더 안정적인 학습을 추구했다. ‘BEGAN(Boundary Equilibrium GAN)’은 WGAN과 EBGAN을 발전시켜 생성하는 이미지의 퀄리티를 획기적으로 높이고 이미지의 퀄리티와 다양성을 컨트롤 할 수 있게 했다. 이 글의 처음에 있는 <그림1>의 실제 같은 사람 얼굴 이미지들이 BEGAN의 결과물이다.

cGAN은 강력한 모델이지만 입력 이미지와 출력 이미지가 매칭된 데이터를 필요로 한다. ‘CycleGAN’과 ‘DiscoGAN’은 두 영역의 이미지 데이터셋이 매칭돼 있지 않아도 이미지 변형을 가능하게 하는 모델이다. 예를 들어 핸드백과 신발의 이미지 데이터셋이 있으면 이 두 영역 사이의 연결을 스스로 찾아내어 주어진 핸드백 이미지와 같은 스타일의 신발 이미지를 생성한다(CycleGAN과 DiscoGAN은 거의 동일한 구조를 갖고 있다). ‘StarGAN’은 이 아이디어를 확장시켜 세 개 이상의 영역 사이의 이미지 변형을 시도했다.

GAN을 다양한 분야에 응용하려는 시도도 활발하다. 사진의 해상도를 높이는 ‘SRGAN(Super-Resolution GAN)’이나, 음성 녹음에서 노이즈를 줄여주는 ‘SEGAN(Speech Enhancement GAN)’을 예로 들 수 있다.

그림11: 핸드백과 같은 스타일의 신발 이미지를 생성하는 DiscoGAN

## GAN의 한계점
GAN은 많은 기대를 받고 있는 모델이지만 아직 여러 가지 한계점도 존재한다. 앞서 소개한 많은 모델이 GAN의 학습을 안정화시키기 위해 노력했지만, 아직도 GAN을 실제로 적용하려 할 때 가장 큰 걸림돌은 학습이 어렵다는 점이다. GAN 학습이 잘 되기 위해서는 서로 비슷한 수준의 생성자와 구분자가 함께 조금씩 발전해야 한다. 그런데 한쪽이 너무 급격하게 강력해지면 이 관계가 깨져버려서 GAN의 학습이 이루어지지 않는다. 경찰이 너무 강력하면 위조지폐범의 씨가 말라버리는 것이다.

GAN이 제대로 학습을 하지 못하고 있을 때 나타나는 모드붕괴(Mode Collapse)라는 현상이 있다. 이는 생성자가 다양한 이미지를 만들어내지 못하고 비슷한 이미지만 계속해서 생성하는 경우를 뜻한다. GAN을 학습시킬 때는 이런 모드 붕괴 현상이 벌어지지 않는지, 생성자와 구분자 중 한 쪽이 너무 강해지지 않는지 유의해야 한다.

GAN의 학습이 너무 어려울 때는 ‘VAE(Variational Auto-Encoder)’라는 모델을 쓰는 것도 고려해 볼 수 있다. VAE는 생성 모델이라는 목적은 GAN과 같지만, GAN과는 다른 방식으로 동작한다. VAE는 GAN보다 학습이 좀 더 안정적이라는 점이 장점이지만, 흐릿한 이미지가 생성되는 블러(Blur) 현상이 있다고 알려져 있다.

GAN의 또 다른 한계점은 아직 텍스트를 생성하는 데는 적용하기 어렵다는 점이다. 이미지나 음성 분야에서는 GAN의 성공 사례가 많이 있지만, 영어나 한국어 같은 자연어를 생성하는 문제에는 GAN의 성공 사례를 찾아보기 어렵다. 이는 텍스트가 이미지와 달리 불연속적이기 때문이다. 이미지는 실수값인 픽셀로 이루어져 있기 때문에 미분을 통해 조금씩 값을 바꿔보며 개선해 나갈 수 있다. 하지만 텍스트는 단어로 이뤄져 있기 때문에 이런 방식이 불가능하다. 물론 GAN으로 텍스트를 생성하려는 시도가 없었던 것은 아니지만, 아직 뚜렷한 성과는 내지 못하고 있다.

마치며#
이안 굿펠로우(Ian Goodfellow)가 GAN을 처음 발표한 것은 2014년이다. 그 후 4년에 가까운 시간이 지나는 동안 GAN에는 수많은 발전이 있었다. 이제 GAN으로 사람 눈으로도 진짜와 구분하기 힘든 얼굴 이미지를 생성하고, 스케치에 채색을 입히고, 핸드백과 같은 스타일의 신발을 만들어낼 수 있다. 그러나 여전히 GAN은 태어난지 얼마 되지 않은 모델이고 앞으로 더욱 놀라운 결과물을 내는 모델들과 서비스들이 발표될 것이다. 이 글이 GAN이라는 혁신적인 모델을 이해하는 데 도움이 되고 인공 지능의 물결에 올라타는 발판이 됐으면 하는 바람이다.

참고 자료#

GAN: https://arxiv.org/abs/1406.2661

DCGAN: https://arxiv.org/abs/1511.06434

cGAN: https://arxiv.org/abs/1611.07004

WGAN: https://arxiv.org/abs/1701.07875

EBGAN: https://arxiv.org/abs/1609.03126

BEGAN: https://arxiv.org/abs/1703.10717

CycleGAN: https://arxiv.org/abs/1703.10593

DiscoGAN: https://arxiv.org/abs/1703.05192

StarGAN: https://arxiv.org/abs/1711.09020

SRGAN: https://arxiv.org/abs/1609.04802

SEGAN: https://arxiv.org/abs/1703.09452
